In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,TargetEncoder,PowerTransformer
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold,RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import optuna
from plotly.io import show
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
df = pd.read_csv('final_data.csv')
df = df.drop(['Title','Price_per_sqft','Is_in_Dhaka'],axis=1)
df.head()

,Bedrooms,Bathrooms,Floor_area,City,location_area,Price_in_Cr,floor_level
0,3.0,4.0,1960.0,dhaka,gulshan,3.90,low
1,3.0,3.0,1705.0,dhaka,kalabagan,1.69,low
2,3.0,3.0,1370.0,dhaka,dhanmondi,1.25,medium
3,3.0,3.0,2125.0,dhaka,bashundhara,2.00,medium
4,3.0,3.0,2687.0,dhaka,banani,4.75,medium


In [4]:
x = df.drop('Price_in_Cr',axis=1)
y = df['Price_in_Cr']

# Optuna 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

floor_area_pipe = Pipeline([
    ('log_transform',PowerTransformer(method='yeo-johnson')),
    ('scalar',StandardScaler())
])

transformer = ColumnTransformer([
    ("floor_area",floor_area_pipe,['Floor_area']),
    ("log_transform",PowerTransformer(method='yeo-johnson'),['Floor_area']),
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
],remainder='passthrough')


def objective(trial,preprocessor):
     regressor_name = trial.suggest_categorical('regressor',['SVR'])

     if regressor_name == 'SVR':
        kernel = trial.suggest_categorical('model__kernel',['rbf','linear','poly','sigmoid'])
        C = trial.suggest_float('model__C',0.01,1000)
        epsilon = trial.suggest_float('model__epsilon', 0.01,1)
        gamma = trial.suggest_categorical('model__gamma', ['scale', 'auto'])

     

        model_pipe = Pipeline([
            ('preprocessing',transformer),
            ('model',SVR(kernel=kernel,C=C,epsilon=epsilon,gamma=gamma))
        ])

                              
        final_pipe = TransformedTargetRegressor(regressor=model_pipe,func=np.log1p,inverse_func=np.expm1)

        trial.set_user_attr('model',final_pipe)

        cv = cross_val_score(final_pipe,x_train,y_train,cv=KFold(n_splits=5,shuffle=True,random_state=0),scoring='neg_mean_squared_error')

        return cv.mean()
     

study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=0),pruner=optuna.pruners.HyperbandPruner())
study.optimize(lambda trial: objective(trial,transformer),n_trials=100,n_jobs=-1)

print(f'Best neg score: {study.best_value}')
print(f'Best Params: {study.best_params}')

final_model = study.best_trial.user_attrs['model']
final_model.fit(x_train,y_train)
y_pred = final_model.predict(x_test)

print(f'MSE: {mean_squared_error(y_test,y_pred)}')
print(f'MAE: {mean_absolute_error(y_test,y_pred)}')
print(f'R2: {r2_score(y_test,y_pred)}')

[I 2025-09-26 17:53:21,671] A new study created in memory with name: no-name-2ba81a85-79f3-459b-89f2-82c3901ba3af
[I 2025-09-26 17:53:22,869] Trial 2 finished with value: -1.671132546303032 and parameters: {'regressor': 'SVR', 'model__kernel': 'poly', 'model__C': 796.1808977908337, 'model__epsilon': 0.9100188794220854, 'model__gamma': 'scale'}. Best is trial 2 with value: -1.671132546303032.
[I 2025-09-26 17:53:22,886] Trial 1 finished with value: -1.4635890311344195 and parameters: {'regressor': 'SVR', 'model__kernel': 'rbf', 'model__C': 85.92401452450841, 'model__epsilon': 0.8324366754129553, 'model__gamma': 'auto'}. Best is trial 1 with value: -1.4635890311344195.
[I 2025-09-26 17:53:22,925] Trial 8 finished with value: -0.24001814978827557 and parameters: {'regressor': 'SVR', 'model__kernel': 'linear', 'model__C': 419.7128157651462, 'model__epsilon': 0.44688745042413325, 'model__gamma': 'auto'}. Best is trial 8 with value: -0.24001814978827557.
[I 2025-09-26 17:53:22,994] Trial 6 f

In [ ]:
final_model

NameError: name 'final_model' is not defined